### Feed Forward

In [8]:
%run core.ipynb

#### Implementation of Feedforward Neural Networks

In [9]:
# %load feedforward.py
import random
import np as _np
import numpy as np


def feedforward(a, N, x):
    for W, B in N:
        x = a(np.dot(W, x) + B)
    return x


def get_random_weights_and_biases(k, l, m=None, layers=0):
    N = [(np.random_matrix(l, k), np.random_vector(l))]
    for _ in range(layers):
        N.append((np.random_matrix(l, l), np.random_vector(l)))
    if m is not None:
        N.append((np.random_matrix(m, l), np.random_vector(m)))
    return N


class FeedForward:

    def __init__(self, dim, layers, a):
        self.dim = dim
        self.N = get_random_weights_and_biases(*dim, layers)
        self.a = a

    def __call__(self, x):
        return self.feedforward(x)

    def __repr__(self):
        return repr(self.N)

    def feedforward(self, x):
        return feedforward(self.a, self.N, x)


class FeedForwardSingleLayer(FeedForward):

    def __init__(self, dim, a, da):
        super().__init__(dim=(*dim, None), layers=0, a=a)
        self.da = da

    def train(self, T, s=0.01, epochs=50):
        for _ in range(epochs):
            for x, y in random.sample(T, len(T)):
                z = feedforward(np.linear, self.N, x)
                az, daz = self.a(z), self.da(z)
                d = y - az
                for W, B in self.N:
                    for i, w in enumerate(W):
                        w += s * d[i] * daz[i] * x
                    B += s * d * daz


In [10]:
a = lambda x: np.multiply(x, 1)
N = [
    (np.matrix((2, 2), 
               (1, -1),
               (2, 2)),  
     np.vector(1, -1, 0)),
    (np.matrix((1, 2, 2),
               (0, 0, 3),
               (1, 2, 2)), 
     np.vector(1, -1, -2)),
    (np.matrix((2, 2, 1),), 
     np.vector(1,)),
]
x = np.vector(1, -1)

feedforward(a, N, x)

array([8])

In [11]:
a = np.sigmoid
N = get_random_weights_and_biases(2, 3, 2, layers=3)
x = np.random_vector(2)
feedforward(a, N, x)

array([0.86491416, 0.87643034])

In [12]:
f = FeedForward(dim=(2, 3, 2), layers=3, a=np.sigmoid)
x = np.random_vector(2)
f(x)

array([0.89047102, 0.86149696])

In [13]:
f = FeedForwardSingleLayer(dim=(3, 2), a=np.sigmoid, da=np.dx_sigmoid)
f

[(array([[0.33818187, 0.77405518, 0.90836759],
       [0.1557715 , 0.3088924 , 0.37986004]]), array([0.48812276, 0.47323944]))]

#### Single Layer Feedforward for Decimal to Binary Converter

In [14]:
from dec2bin import int2dec, int2bin, bin2int


class Dec2Bin(FeedForwardSingleLayer):

    def __init__(self, a=np.sigmoid, da=np.dx_sigmoid, 
                 classify=lambda x: heaviside(x, offset=0.5), s=0.01, epochs=100):
        super().__init__(dim=(10, 4), a=a, da=da)
        self.classify = classify
        self.train([(np.vector(*int2dec(n)), np.vector(*int2bin(n))) for n in range(10)], s, epochs)

    def __call__(self, n):
        return tuple([self.classify(t) for t in super().__call__(int2dec(n))])

    
def run_dec2bin_feedforward(f):
    for n in range(10):
        y = f(n)
        z = tuple(int2bin(n))
        if y == z:
            print(f"{n} = {y} {_ok}")
        else:
            print(f"{n} = {z} ≠ {_red(y)} {_nok}")
            

In [15]:
a, da, classify = np.sigmoid, np.dx_sigmoid, lambda x: heaviside(x, offset=0.5)
s, epochs = 0.01, 1500
f = Dec2Bin(a=a, da=da, classify=classify, s=s, epochs=epochs)

run_dec2bin_feedforward(f)

0 = (0, 0, 0, 0) ✔
1 = (1, 0, 0, 0) ✔
2 = (0, 1, 0, 0) ✔
3 = (1, 1, 0, 0) ✔
4 = (0, 0, 1, 0) ✔
5 = (1, 0, 1, 0) ✔
6 = (0, 1, 1, 0) ✔
7 = (1, 1, 1, 0) ✔
8 = (0, 0, 0, 1) ✔
9 = (1, 0, 0, 1) ✔


In [21]:
a, da, classify = np.relu, np.dx_relu, lambda x: heaviside(x, offset=0.5)
s, epochs = 0.01, 150
f = Dec2Bin(a=a, da=da, classify=classify, s=s, epochs=epochs)

run_dec2bin_feedforward(f)

0 = (0, 0, 0, 0) ✔
1 = (1, 0, 0, 0) ✔
2 = (0, 1, 0, 0) ✔
3 = (1, 1, 0, 0) ✔
4 = (0, 0, 1, 0) ✔
5 = (1, 0, 1, 0) ✔
6 = (0, 1, 1, 0) ✔
7 = (1, 1, 1, 0) ✔
8 = (0, 0, 0, 1) ✔
9 = (1, 0, 0, 1) ✔


#### Single Layer Feedforward for Boolean Operator

In [17]:
class BooleanOperator(FeedForwardSingleLayer):

    def __init__(self, operator, a=np.sigmoid, da=np.dx_sigmoid, 
                 classify=lambda x: np.heaviside(x, offset=0.5), s=0.01, epochs=50):
        super().__init__(dim=(2, 1), a=a, da=da)
        self.classify = classify
        self.operator = operator
        self.train([(np.vector(*t), np.vector(y)) for t, y in BOOLEAN_TESTDATA[operator]], s, epochs)

    def __call__(self, x, y):
        return self.classify(super().__call__(np.vector(x, y))[0])


def run_boolean_feedforward(f):
    for t, y in BOOLEAN_TESTDATA[f.operator]:
        z = f.feedforward(np.vector(*t))[0]
        if y == f.classify(z):
            print(f"{f.operator.__name__}{t} = {y} ({_rnd(z)}) {_ok}")
        else:
            print(f"{f.operator.__name__}{t} = {y} ≠ {_rnd(z)} {_nok}")
    with np.printoptions(precision=2, suppress=True):                
        print(f"{f.operator.__name__}.N = {f.N}")


def verify_boolean_feedforward(operator, a=np.sigmoid, da=np.dx_sigmoid, classify=lambda x: heaviside(x, offset=0.5), 
                               s=0.1, epochs=50, trials=25):
    for trial in range(trials):
        f = BooleanOperator(operator, a=a, da=da, classify=classify, s=s, epochs=epochs)
        if not all(y == f(*t) for t, y in BOOLEAN_TESTDATA[f.operator]):
            print(f"{operator.__name__}: {trial}/{trials} trials {_nok}")
            run_boolean_feedforward(f)
            return
    print(f"{operator.__name__}:	{trials} trials {_ok}")


In [18]:
a, da, classify = np.relu, np.dx_relu, lambda x: heaviside(x, offset=0.4)
s, epochs = 0.1, 250
f = BooleanOperator(OR, a=a, da=da, classify=classify, s=s, epochs=epochs)

run_boolean_feedforward(f)

OR(0, 0) = 0 (0.25) ✔
OR(0, 1) = 1 (0.73) ✔
OR(1, 0) = 1 (0.73) ✔
OR(1, 1) = 1 (1.21) ✔
OR.N = [(array([[0.48, 0.48]]), array([0.25]))]


In [19]:
a, da, classify = np.tanh, np.dx_tanh, lambda x: heaviside(x, offset=0.5)
a, da, classify = np.sigmoid, np.dx_sigmoid, lambda x: heaviside(x, offset=0.5)
s, epochs = 0.1, 250
operators = [OR, AND, NAND]
trials = 100
for operator in operators:
    verify_boolean_feedforward(operator, a=a, da=da, classify=classify, s=s, epochs=epochs, trials=trials)

OR:	100 trials ✔
AND:	100 trials ✔
NAND:	100 trials ✔


#### NAND Approximation using ReLu

In [20]:
a, da, classify = np.relu, np.dx_relu, lambda x: heaviside(x, offset=0.4)
s, epochs = 0.1, 50
trials = 100
verify_boolean_feedforward(NAND, a=a, da=da, classify=classify, s=s, epochs=epochs, trials=trials)

NAND: 0/100 trials ✗
NAND(0, 0) = 1 ≠ 0.0 ✗
NAND(0, 1) = 0 (0.0) ✔
NAND(1, 0) = 0 (0.0) ✔
NAND(1, 1) = 0 (0.0) ✔
NAND.N = [(array([[-0.04,  0.12]]), array([-0.12]))]
